Authors:
Patrice Carbonneau and Simone Bizzi

License: MIT

In [ ]:
#Imports
import numpy as np
import pandas as pd
import random
from osgeo import gdal
import glob
import os
import time

#Import earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=08QuHK2VIOHNLpfAaXqtx7Sf_ki4_m0LIgP1Jp52axY&tc=jCJ-0I_Uao0amC8NuWWbOBd7aMuXWsXt1UGwwMMg8GU&cc=oZuUz4Iw41VcHsZzTxzspvax_RTjbzQ6csOj3ypIZUo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VPJDv3PNGorqr7_Jl7dYGwnH5rRC62systFB9PhltgzwoNxEMwTiPI

Successfully saved authorization token.


In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
#Function definitions

#Cloud Mask functions and parameters

CLOUD_FILTER = 20
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 10

def get_s2_sr_cld_col(AOI, date1, date2):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(AOI)
        .filterDate(date1, date2)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        )

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(AOI)
        .filterDate(date1, date2))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()
    cld_shdw = img.select('cloudmask')

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


def GDALpix2map(ds, x, y):
    xoffset, px_w, rot1, yoffset, rot2, px_h = ds.GetGeoTransform()
    posX = px_w * x + rot1 * y + xoffset
    posY = rot2 * x + px_h * y + yoffset
    # shift to the center of the pixel
    posX += px_w / 2.0
    posY += px_h / 2.0
    return posX, posY


def GetCRS(letter, zone):

  North=['N','P', 'Q', 'R','S','T','U','V','W', 'X']

  if letter in North:
    thisCRS='EPSG:326'+str(zone).zfill(2)
  else:
    thisCRS='EPSG:327'+str(zone).zfill(2)

  return thisCRS

def GetFalseCRS(letter, zone):

  thisCRS='EPSG:32618'

  return thisCRS



def GetGZD(AOI, zone, letter, MonthList, year, thisCRS, NormFactor):
  for m in range(len(MonthList)):
    month=MonthList[m]
    month1=f'{month:02d}'
    month2=f'{month+1:02d}'
    date1=str(year)+"-"+month1+"-"+"01"
    if int(month2)>12:

      month2=str(int(month2)-12).zfill(2)
      #year=year+1
    #date2=str(year)+"-"+month2+"-"+"01"#10 month window
    if True:
      if month==1:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"
      elif month==2:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"28"
      elif month==3:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"
      elif month==4:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"30"
      elif month==5:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"
      elif month==6:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"30"
      elif month==7:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"
      elif month==8:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"
      elif month==9:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"30"
      elif month==10:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"
      elif month==11:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"30"
      elif month==12:
        date1=str(year)+"-"+month1+"-"+"01"
        date2=str(year)+"-"+month1+"-"+"31"




    print(date1+' to '+date2)
    s2_sr_cld_col = get_s2_sr_cld_col(AOI, date1, date2)
    s2_sr_masked = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             )

    #S2c = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(AOI).filterDate(date1, date2).filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',10)
    S2r=s2_sr_cld_col.select('B8', 'B4', 'B3').reduce(ee.Reducer.median())
    S2i=ee.Image(S2r).divide(NormFactor).multiply(255).uint8()#convert to 0-255 range to reduce download volume

    #get clouds
    S2mask=s2_sr_masked.select('B3').reduce(ee.Reducer.median())
    CloudMask=ee.Image(S2mask).uint8().rename('Clouds')

    #get metadata. Normal data=> all bands 255, clouds=> Metadata 0 others non-0, nodata=> all 0
    mask=s2_sr_cld_col.select('B2').reduce(ee.Reducer.sum())
    maski=ee.Image(mask).multiply(CloudMask).uint8().rename('METADATA')
    newBands = ee.Image([maski])
    S2i=S2i.addBands(newBands)
    #reproject output to UTM
    S2iUTM=S2i.reproject(thisCRS, scale=10)

    S2Name='S2_M'+str(month).zfill(2)+"_Z"+str(zone).zfill(2)+letter


    task = ee.batch.Export.image.toDrive(**{
    'image': S2iUTM,
    'description': S2Name,
    'scale': 10,
    'region':AOI,
    'folder':"GEES2_"+str(year)+'M'+str(month).zfill(2)+"_Z"+str(zone).zfill(2)+letter,
    'maxPixels': 1e13,
    })

    task.start()

    time.sleep(1)
    '''add a second task that will export a MODIS snow mask layer'''
    SnowColl = ee.ImageCollection('MODIS/006/MOD10A1').filterBounds(AOI).filter(ee.Filter.date(date1, date2))
    snowCover = SnowColl.select('NDSI_Snow_Cover').reduce(ee.Reducer.median())
    Snow = snowCover.gte(50).rename('Snow')
    SnowUTM=Snow.reproject(thisCRS, scale=250)
    SnowName='SNOW_'+ S2Name#[3:]

    snowtask = ee.batch.Export.image.toDrive(**{
    'image': SnowUTM,
    'description': SnowName,
    'scale': 250,
    'region':AOI,
    'folder':"GEESNOW_"+str(year)+'M'+str(month).zfill(2),#"GEE_Meta_M"+str(month).zfill(2),#+"_Z"+str(zone).zfill(2)+letter,
    'maxPixels': 1e13,
    })

    snowtask.start()





In [ ]:
#Extract mosaics with calls to GetUTMletterAnnual, get the AOI from a dataframe with a view to looping through the world
Part='C'
Month=[8] #enter the month needed here, can only enter 1
year=2019

#different parts can be used to spread the download over batches
if 'A' in Part:
  StartLine=0
  StopLine=225
elif 'B' in Part:
  StartLine=169
  StopLine= 355
elif 'C' in Part:
  StartLine=0 #whole sheet
  StopLine=-1



GlobalBoundsFile=''#Enter a csv file with the bounds required
boundsfolder='/content/drive/MyDrive/ColabData/BoundFiles/'#located on Drive, adjust if needed
boundsDF=pd.read_csv(boundsfolder+GlobalBoundsFile)
NormFactor=10000 #used to convert int16 to 0-1 before *255 for uint8 conversion

if StopLine<0:
  StopLine=len(boundsDF.index)


for DFline in range(StartLine,StopLine):
  zone= np.asarray(boundsDF.iloc[DFline].Zone)
  letter = boundsDF.iloc[DFline].GZD
  thisCRS=GetCRS(letter, zone)

  if zone % 2 > -1: #can use this to select even or odd or all zones
    bounds=np.asarray([boundsDF.iloc[DFline].LLx, boundsDF.iloc[DFline].LLy, boundsDF.iloc[DFline].URx, boundsDF.iloc[DFline].URy ])
    print(bounds) #these will be in latlon for GEE convenience
    AOI=ee.Geometry.Rectangle([bounds[0], bounds[1], bounds[2], bounds[3]])
    GetGZD(AOI, zone, letter, Month, year,thisCRS, NormFactor)# crazy big export to drive
    print('Processing Zone'+str(zone)+letter)
    time.sleep(0.1)



[-179.5    64.   -173.95   72.01]
2019-08-01 to 2019-08-31
Processing Zone1W
[-174.     64.   -167.95   72.01]
2019-08-01 to 2019-08-31
Processing Zone2W
[-168.     48.   -161.95   56.01]
2019-08-01 to 2019-08-31
Processing Zone3U
[-168.     56.   -161.95   64.01]
2019-08-01 to 2019-08-31
Processing Zone3V
[-168.     64.   -161.95   72.01]
2019-08-01 to 2019-08-31
Processing Zone3W
[-162.     48.   -155.95   56.01]
2019-08-01 to 2019-08-31
Processing Zone4U
[-162.     56.   -155.95   64.01]
2019-08-01 to 2019-08-31
Processing Zone4V
[-162.     64.   -155.95   72.01]
2019-08-01 to 2019-08-31
Processing Zone4W
[-156.     56.   -149.95   64.01]
2019-08-01 to 2019-08-31
Processing Zone5V
[-156.     64.   -149.95   72.01]
2019-08-01 to 2019-08-31
Processing Zone5W
[-150.     56.   -143.95   64.01]
2019-08-01 to 2019-08-31
Processing Zone6V
[-150.     64.   -143.95   72.01]
2019-08-01 to 2019-08-31
Processing Zone6W
[-144.     56.   -137.95   64.01]
2019-08-01 to 2019-08-31
Processing Zone7V


##UTM ZONES with MGRS letters

<img src='https://drive.google.com/uc?id=1ZcAf1Ixh5XliYluBogcg6xj0Fz4ng46P' width=1000>

In [ ]:
#GEE task listing cell.  usefull to check progress
from subprocess import PIPE, Popen

blunders=300
command = 'earthengine task list'
with Popen(command, stdout=PIPE, stderr=None, shell=True) as process:
    output = process.communicate()[0].decode("utf-8")
    print(output)



3BZZCCJPRDP27YINMSJR3ZF3  Export.image  SNOW_S2_M08_Z33T  READY      ---
H4L7VGCZKPTIGBK4UBFT4UCI  Export.image  S2_M08_Z33T       READY      ---
RWUY7NBSIV6VLYVRMZC6WVUG  Export.image  SNOW_S2_M08_Z33S  READY      ---
ASHO6GPDQS27UWOUS3JIBKCV  Export.image  S2_M08_Z33S       READY      ---
ZXLWZPEAO3SGF7UKLNN62UD5  Export.image  SNOW_S2_M08_Z32T  READY      ---
GWCK5PZC2HK2JEOXHQUPDHPL  Export.image  S2_M08_Z32T       READY      ---
LROPMTPB5ZHND2WOTGN5IYJG  Export.image  SNOW_S2_M08_Z32S  READY      ---
2LCCHMTRCGMF7LUH5A3BLJ45  Export.image  S2_M08_Z32S       READY      ---
QH6LER5K35WI3FKIEJSWBKMD  Export.image  SNOW_S2_M08_Z33T  READY      ---
5KKCRKQFUEZMM6JAYSI27IM5  Export.image  S2_M08_Z33T       RUNNING    ---
73XENCTI2FGYWISX3U5PMH5L  Export.image  SNOW_S2_M08_Z33S  COMPLETED  ---
STM5NKPBJE4YOKN722SYIXCQ  Export.image  S2_M08_Z33S       RUNNING    ---
QS46PZMI2TSHT72BTH3QLMAS  Export.image  SNOW_S2_M08_Z32T  COMPLETED  ---
LD5D2YP2YO2HMNVKRV4LCBGE  Export.image  S2_M08_Z32T

In [ ]:
#flush all GEE tasks
import os
command='earthengine task cancel all'
os.system(command)

0